In [2]:
import pyspark
from mmlspark.RankingEvaluator import RankingEvaluator
from pyspark.ml.tuning import *
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS
from mmlspark.RankingTrainValidationSplit import RankingTrainValidationSplit

In [3]:
def getRatings():
        cSchema = StructType([StructField("user", IntegerType()),
                              StructField("item", IntegerType()),
                              StructField("rating", IntegerType()),
                              StructField("notTime", IntegerType())])

        ratings = pyspark.sql.SparkSession.builder.getOrCreate().createDataFrame([
            (0, 1, 4, 4),
            (0, 3, 1, 1),
            (0, 4, 5, 5),
            (0, 5, 3, 3),
            (0, 7, 3, 3),
            (0, 9, 3, 3),
            (0, 10, 3, 3),
            (1, 1, 4, 4),
            (1, 2, 5, 5),
            (1, 3, 1, 1),
            (1, 6, 4, 4),
            (1, 7, 5, 5),
            (1, 8, 1, 1),
            (1, 10, 3, 3),
            (2, 1, 4, 4),
            (2, 2, 1, 1),
            (2, 3, 1, 1),
            (2, 4, 5, 5),
            (2, 5, 3, 3),
            (2, 6, 4, 4),
            (2, 8, 1, 1),
            (2, 9, 5, 5),
            (2, 10, 3, 3),
            (3, 2, 5, 5),
            (3, 3, 1, 1),
            (3, 4, 5, 5),
            (3, 5, 3, 3),
            (3, 6, 4, 4),
            (3, 7, 5, 5),
            (3, 8, 1, 1),
            (3, 9, 5, 5),
            (3, 10, 3, 3)], cSchema)
        return ratings

In [4]:
ratings = getRatings()

als = ALS() \
    .setUserCol("user") \
    .setRatingCol('rating') \
    .setItemCol("item") \

paramGrid = ParamGridBuilder() \
    .addGrid(als.maxIter, [1, 2, 3, 5, 8]) \
    .build()

evaluator = RankingEvaluator()

rankingTrainValidationSplit = RankingTrainValidationSplit() \
    .setEstimator(als) \
    .setEvaluator(evaluator) \
    .setEstimatorParamMaps(paramGrid) \
    .setTrainRatio(0.8) \
    .setCollectSubMetrics(True)

model = rankingTrainValidationSplit.fit(data_all)

usersRecs = model.recommendForAllUsers(3)

print("User Recs: ")
print(usersRecs.take(1))
print("Validation Metrics: ")
print(model.validationMetrics)

metrics = evaluator._call_java("getMetricsList").toString()
print("Metrics List: ")
print(metrics)

fit starting
User Recs: 
[Row(user=1, recommendations=[Row(item=7, rating=4.925848007202148), Row(item=2, rating=4.852099418640137), Row(item=4, rating=4.692464351654053)])]
Validation Metrics: 
[0.3086598407528446, 0.38268031849431083, 0.38268031849431083, 0.42597952225853364, 0.3519590445170674]
Metrics List: 
ListBuffer(Map(map -> 0.2222222222222222, maxDiversity -> -10.0, diversityAtK -> -9.0, recallAtK -> 0.3333333333333333, ndcgAt -> 0.3086598407528446, precisionAtk -> 0.3333333333333333), Map(map -> 0.2638888888888889, maxDiversity -> -8.0, diversityAtK -> -7.0, recallAtK -> 0.41666666666666663, ndcgAt -> 0.38268031849431083, precisionAtk -> 0.41666666666666663), Map(map -> 0.2638888888888889, maxDiversity -> -8.0, diversityAtK -> -7.0, recallAtK -> 0.41666666666666663, ndcgAt -> 0.38268031849431083, precisionAtk -> 0.41666666666666663), Map(map -> 0.3055555555555556, maxDiversity -> -7.0, diversityAtK -> -5.0, recallAtK -> 0.41666666666666663, ndcgAt -> 0.42597952225853364, precisionAtk -> 0.41666666666666663), Map(map -> 0.2222222222222222, maxDiversity -> -7.0, diversityAtK -> -3.0, recallAtK -> 0.3333333333333333, ndcgAt -> 0.3519590445170674, precisionAtk -> 0.3333333333333333))